<a href="https://colab.research.google.com/github/TonioFlag/number_check_whatsapps/blob/main/version_google_colab/verificador_whatsapps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium qrcode -q
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
!apt-get install chromium-driver -q
import random
import time
import qrcode
import IPython.display as display
from IPython.display import clear_output
from PIL import Image
import pandas as pd
import os
import sys
from datetime import datetime, timedelta

class VerificarWhatsApp:

  def __init__(self):
    clear_output(wait=True)
    options = webdriver.FirefoxOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    self.driver = webdriver.Firefox(options=options)
    ruta_archivo = 'base.csv'
    if not os.path.isfile(ruta_archivo):
      print(f"Error: El archivo '{ruta_archivo}' no se encuentra.")
      print("Por favor asegúrese de que el archivo esté en la misma carpeta que el script de Python.")
      sys.exit()

    self.base = pd.read_csv(ruta_archivo)

    columnas_archivo = self.base.columns.tolist()
    columnas_esperadas = ["Móvil", "Está en Whatsapp?", "Fecha"]

    self.base['Fecha'] = self.base['Fecha'] = pd.to_datetime(self.base['Fecha'], errors='coerce')

    if columnas_archivo != columnas_esperadas:
      print("Las columnas no coinciden con las esperadas.")
      print(f"Columnas encontradas: {columnas_archivo}")
      print(f"Columnas esperadas: {columnas_esperadas}")
      sys.exit()


  def get_qr(self,i):
    if (i<6):
      clear_output(wait=True)
      self.driver.get('https://web.whatsapp.com')
      try:
        time.sleep(8)
        element = self.driver.find_element(By.CLASS_NAME, "_akau.x1n2onr6.x78zum5.x1okw0bk.x6s0dn4.xl56j7k.x1tdqgrh.x1l76qip.x6ikm8r.x10wlt62.xm3z3ea.x1x8b98j.x131883w.x16mih1h")
        data_ref_value = element.get_attribute('data-ref')
        qr_img = qrcode.make(data_ref_value)
        qr_img = qr_img.resize((264, 264))
        print(f"Escanea el QR para empezar el proceso. Intento {i}/5.")
        display.display(qr_img)
        time.sleep(25)
        clear_output(wait=True)
        if self.is_loggin():
          print("Sesion iniciada con éxito.")
        else:
          self.get_qr(i+1)

      except Exception as e:
        if self.is_loggin():
          print("Sesion iniciada con éxito.")
          print("Iniciando proceso...")
        else:
          self.get_qr(i+1)
    else:
      print("No se pudo iniciar la sesión.")
      sys.exit()

  def is_loggin(self):
    try:
      clear_output(wait=True)
      time.sleep(5)
      element = self.driver.find_element(By.XPATH, '//*[@id="side"]/div[1]/div/div[2]/div[1]')
      if element.text == "Search" or element.text == "Buscar":
        return True
      else:
        return False
    except Exception as e:
      return False

  def number_check(self):
    time.sleep(2)
    clear_output(wait=True)
    print("Procesando números de WhatsApp, esto tomará un tiempo... \n")
    url_izquierda = 'https://web.whatsapp.com/send/?phone=%2B'
    url_derecha = '&text&type=phone_number&app_absent=0'

    for index,row in self.base.iterrows():
      i = row['Móvil']
      fecha = row['Fecha']
      tiene = row['Está en Whatsapp?']
      hoy = datetime.now()
      limite = hoy - timedelta(weeks=52/2)

      if fecha<limite or tiene == "Error" or tiene == "" or pd.isnull(fecha):
        url = url_izquierda + str(i).replace("+","").replace(" ","") + url_derecha
        self.driver.get(url)
        WebDriverWait(self.driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "x12lqup9.x1o1kx08")))
        self.base.at[index, 'Fecha'] = hoy
        time.sleep(1.5)
        resultado = self.exist(1)
        self.base.at[index, 'Está en Whatsapp?'] = resultado
        print(resultado)

      if (index + 1) % 100 == 0:
        self.base.to_csv(f"base_{(index // 100) + 1}.csv", index=False)

    clear_output(wait=True)
    self.base.to_csv('base.csv', index=False)
    print("Proceso finalizado.")

  def exist(self, i):
    if (i<3):
      text1 = "El número de teléfono compartido a través de la dirección URL no es válido."
      text2 = "The phone number shared via the URL is invalid."
      script = """
                function checkNumber () {
                  try{
                    const res = document.getElementsByClassName("x12lqup9 x1o1kx08")[0].textContent
                    if (res == ""){
                      return "Si"
                    }else{
                      return "No"
                    }
                  }catch (err){
                    return "No"
                  }
                }

                return checkNumber()
                    """
      try:
        res = self.driver.execute_script(script)
        return res
      except Exception as e:
        print(f"Error {e}")
        time.sleep(1)
        return self.exist(i+1)
    else:
      return "Error"

  def logout(self):
    try:
      logout_button = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[3]/div[3]/header/header/div/span/div/span/div[2]/span/div/ul/li[5]")
      logout_button.click()
      time.sleep(2)
      logout_button = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div[2]/div/button[2]")
      logout_button.click()
      time.sleep(8)
      print("Se cerro la sesión correctamente.")
      self.driver.quit()
    except Exception as e:
      print("Error al hacer clic en el botón de 'Cerrar sesión':", e)

  def open_menu(self):
    self.driver.get('https://web.whatsapp.com')
    time.sleep(8)
    try:
      menu_button =  self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[3]/div[3]/header/header/div/span/div/span/div[2]/div")
      menu_button.click()
    except Exception as e:
      print("Error al hacer clic en el botón de menú:", e)

scrape = VerificarWhatsApp()
scrape.get_qr(1)
scrape.number_check()
scrape.open_menu()
scrape.logout()
scrape.driver.quit()